In [1]:
import cv2
import numpy as np
import time
import os
from datetime import datetime

# ===============================
# LOAD FILE MODEL
# ===============================
prototxt = "deploy.prototxt"
model = "mobilenet_iter_73000.caffemodel"
labels_path = "labels.txt"

# Load labels
with open(labels_path, "r") as f:
    classes = f.read().strip().split("\n")

# Load model
net = cv2.dnn.readNetFromCaffe(prototxt, model)

# ===============================
# PREPARE SAVE FOLDER
# ===============================
save_dir = "cropped"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# List untuk menyimpan class yang sudah pernah disimpan (anti duplikat)
saved_classes = set()

# ===============================
# VIDEO CAPTURE
# ===============================
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

prev_frame_time = 0

print("Tekan 'q' untuk keluar.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Camera error!")
        break

    # Mirror
    frame = cv2.flip(frame, 1)

    (h, w) = frame.shape[:2]

    # Create blob
    blob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)),
        0.007843, (300, 300),
        127.5
    )
    net.setInput(blob)
    detections = net.forward()

    highest_conf = 0
    best_crop = None
    best_label = None

    object_count = 0

    # ============ PROCESS DETECTION ============
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.45:
            idx = int(detections[0, 0, i, 1])
            label = classes[idx]

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")

            object_count += 1

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 180, 255), 2)
            cv2.putText(frame, f"{label} {confidence*100:.1f}%",
                        (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 180, 255), 2)

            # Tentukan crop yang paling kuat confidence
            if confidence > highest_conf:
                highest_conf = confidence
                best_crop = frame[y1:y2, x1:x2]
                best_label = label

    # ============ FPS ============
    new_frame_time = time.time()
    fps = 1 / (new_frame_time - prev_frame_time + 1e-8)
    prev_frame_time = new_frame_time

    cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.putText(frame, f"Objects Detected: {object_count}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    # ============ SAVE CROP (1x per class) ============
    if best_crop is not None:
        if best_label not in saved_classes:
            filename = f"{best_label}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
            path = os.path.join(save_dir, filename)

            cv2.imwrite(path, best_crop)
            print(f"> Saved FIRST detection of {best_label} → {path}")

            saved_classes.add(best_label)

    # ============ TAMPILKAN 1 WINDOW SAJA ============
    cv2.imshow("Object Detection — Dea", frame)

    # Tekan q untuk keluar
    key = cv2.waitKey(1)
    if key & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

Tekan 'q' untuk keluar.
> Saved FIRST detection of pottedplant → cropped\pottedplant_20251229_182141.jpg
> Saved FIRST detection of cat → cropped\cat_20251229_182141.jpg
> Saved FIRST detection of person → cropped\person_20251229_182142.jpg
> Saved FIRST detection of diningtable → cropped\diningtable_20251229_182144.jpg
> Saved FIRST detection of bottle → cropped\bottle_20251229_182145.jpg
